In [1]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 29.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

ds = load_dataset("SPRINGLab/IndicTTS_Telugu")

print(ds)

README.md:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8576 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'gender'],
        num_rows: 8576
    })
})


In [4]:
from datasets import load_dataset, Audio

ds = ds.cast_column("audio", Audio(sampling_rate=16000))

In [5]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [6]:
tokenizer = processor.tokenizer

In [7]:
ds['train'][0:5]

{'audio': [{'path': 'train_telugufemale_00001.wav',
   'array': array([0.00082397, 0.00079346, 0.00082397, ..., 0.        , 0.        ,
          0.        ]),
   'sampling_rate': 16000},
  {'path': 'train_telugufemale_00002.wav',
   'array': array([0.00036621, 0.00024414, 0.00024414, ..., 0.        , 0.        ,
          0.        ]),
   'sampling_rate': 16000},
  {'path': 'train_telugufemale_00003.wav',
   'array': array([-0.00076294, -0.00076294, -0.00079346, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 16000},
  {'path': 'train_telugufemale_00004.wav',
   'array': array([0.00054932, 0.00048828, 0.00054932, ..., 0.        , 0.        ,
          0.        ]),
   'sampling_rate': 16000},
  {'path': 'train_telugufemale_00005.wav',
   'array': array([-0.00100708, -0.00094604, -0.00094604, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 16000}],
 'text': ['నరకవాసి.',
  'బ్రహ్మదత్తుడు కాశీరాజ్యాన్ని పరిపాలించే కాలంలో ఆ నగరంలో ధన

In [8]:
def extract_all_chars(batch):
    vocab = set(char for text in batch["text"] for char in text)
    all_text = " ".join(batch["text"])
    return {
        "vocab": [vocab] * len(batch["text"]),
        "all_test": [all_text] * len(batch["text"])
    }

vocabs = ds.map(
    extract_all_chars,
    batched=True,
    batch_size=1000,  # Adjust batch_size as needed
)


Map:   0%|          | 0/8576 [00:00<?, ? examples/s]

In [9]:
ds_vocab = set(vocabs['train']['vocab'][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

ds_vocab - tokenizer_vocab

{' ',
 'ం',
 'ః',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఋ',
 'ఎ',
 'ఏ',
 'ఐ',
 'ఒ',
 'ఓ',
 'ఔ',
 'క',
 'ఖ',
 'గ',
 'ఘ',
 'ఙ',
 'చ',
 'ఛ',
 'జ',
 'ఞ',
 'ట',
 'ఠ',
 'డ',
 'ఢ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ఱ',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ',
 'ా',
 'ి',
 'ీ',
 'ు',
 'ూ',
 'ృ',
 'ె',
 'ే',
 'ై',
 'ొ',
 'ో',
 'ౌ',
 '్'}

In [10]:
replacements = [
  ("అ", "a"),
  ("ఆ", "aa"),
  ("ఇ", "i"),
  ("ఈ", "ee"),
  ("ఉ", "u"),
  ("ఊ", "oo"),
  ("ఋ", "ru"),
  ("ఎ", "e"),
  ("ఏ", "ee"),
  ("ఐ", "ai"),
  ("ఒ", "o"),
  ("ఓ", "oa"),
  ("ఔ", "au"),
  ("క", "ka"),
  ("ఖ", "kha"),
  ("గ", "ga"),
  ("ఘ", "gha"),
  ("ఙ", "nga"),
  ("చ", "cha"),
  ("ఛ", "chha"),
  ("జ", "ja"),
  ("ఞ", "nya"),
  ("ట", "ta"),
  ("ఠ", "tha"),
  ("డ", "da"),
  ("ఢ", "dha"),
  ("ణ", "na"),
  ("త", "tha"),
  ("థ", "thha"),
  ("ద", "da"),
  ("ధ", "dha"),
  ("న", "na"),
  ("ప", "pa"),
  ("ఫ", "pha"),
  ("బ", "ba"),
  ("భ", "bha"),
  ("మ", "ma"),
  ("య", "ya"),
  ("ర", "ra"),
  ("ఱ", "rra"),
  ("ల", "la"),
  ("ళ", "lla"),
  ("వ", "va"),
  ("శ", "sha"),
  ("ష", "shha"),
  ("స", "sa"),
  ("హ", "ha"),
  ("ా", "aa"),
  ("ి", "i"),
  ("ీ", "ee"),
  ("ు", "u"),
  ("ూ", "oo"),
  ("ృ", "ru"),
  ("ె", "e"),
  ("ే", "ee"),
  ("ై", "ai"),
  ("ొ", "o"),
  ("ో", "oa"),
  ("ౌ", "au"),
  ("ం", "m"),
  ("ః", "h"),
  ("్", ""),  # Removes inherent vowel
  (" ", " ")  # Space remains unchanged
]

def cleanup_text(inputs):
  for src, dst in replacements:
    inputs["text"] = inputs["text"].replace(src, dst)
  return inputs

ds = ds.map(cleanup_text)


Map:   0%|          | 0/8576 [00:00<?, ? examples/s]

In [11]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp",spk_model_name)
)

def create_speaker_embedding(waveform):
  with torch.no_grad():
    speaker_embedding = speaker_model.encode_batch(torch.tensor(waveform))
    speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
    speaker_embedding = speaker_embedding.squeeze().cpu().numpy()
  return speaker_embedding

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [12]:
def prepare_dataset(example):
    audio = example["audio"]

    # Truncate text to the maximum sequence length
    text = example["text"][:processor.tokenizer.model_max_length]

    example = processor(
        text=text,
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
        max_length=processor.tokenizer.model_max_length, # Explicitly set max_length
        truncation=True,  # Enable truncation
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [13]:
processed_example = prepare_dataset(ds['train'][0])
list(processed_example.keys())

print(processed_example["speaker_embeddings"].shape)

(512,)


In [14]:
ds = ds.map(prepare_dataset, remove_columns=['audio', 'text', 'gender'])

Map:   0%|          | 0/8576 [00:00<?, ? examples/s]

In [15]:
train_testvalid_ds = ds["train"].train_test_split(test_size=0.1)
ds['train'] = train_testvalid_ds['train']
ds['test'] = train_testvalid_ds['test']

In [16]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

data_collator = TTSDataCollatorWithPadding(processor=processor)

In [17]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [18]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [19]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_telugu_charan",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-3,
    warmup_steps=100,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-cb88b4de2e6e>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-67e45d01-76aa30dd0eb93b0f7a8c3b1c;f2c4fdf6-900d-446e-96c8-61f4828346e3)

Invalid username or password.

In [ ]:
trainer.train()

trainer.push_to_hub()

In [ ]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "Epikwhale/speecht5_finetuned_telugu_charan"
)

In [ ]:
example = ds["test"][16]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

text = "తెలుగు భాష అందమైన శబ్దంతో గొప్ప సంప్రదాయంతో విస్తారమైన సాహిత్యంతో లోకానికి వెలుగు చిందిస్తుంది"

import re

# Mapping for Telugu number words
number_words = {
    0: "సున్నా",
    1: "ఒకటి",
    2: "రెండు",
    3: "మూడు",
    4: "నాలుగు",
    5: "ఐదు",
    6: "ఆరు",
    7: "ఏడు",
    8: "ఎనిమిది",
    9: "తొమ్మిది",
    10: "పది",
    11: "పదకొండు",
    12: "పన్నెండు",
    13: "పదమూడు",
    14: "పద్నాలుగు",
    15: "పదిహేను",
    16: "పదహారు",
    17: "పదిఏడు",
    18: "పద్దెనిమిది",
    19: "పద్దొమ్మది",
    20: "ఇరవై",
    30: "ముప్పై",
    40: "నలభై",
    50: "యాభై",
    60: "అరవై",
    70: "డెబ్బై",
    80: "ఎనభై",
    90: "తొంభై",
    100: "వంద",
    1000: "వెయ్యి"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + (" " + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        # In Telugu, "వంద" is used for 100, and for multiples > 1 we prepend the number word.
        hundreds_word = (number_words[hundreds] + " వంద" if hundreds > 1 else "వంద")
        return hundreds_word + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        thousands_word = (number_to_words(thousands) + " వెయ్యి" if thousands > 1 else "వెయ్యి")
        return thousands_word + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " మిలియన్" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " బిలియన్" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Replace all numbers in the text with their Telugu words
    result = re.sub(r'\b\d+\b', replace, text)
    return result

def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = cleaned_text
print(final_text)



In [ ]:
inputs = processor(text=final_text, return_tensors="pt")

from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [ ]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'telugu.wav')
sf.write('telugu4.wav', speech.numpy(), 16000)

